In [1]:
import os
import json
import math
import numpy as np
from scipy.spatial.transform import Rotation

In [2]:
SAVE_PATH = os.path.join(os.getcwd(), '../data/jackal_hemisphere')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
if not os.path.exists(os.path.join(SAVE_PATH, 'images')):
    os.makedirs(os.path.join(SAVE_PATH, 'images'))

In [3]:
transforms = {}
transforms["focal_length"] = 50.0 #Focal Length in mm
transforms["fov_x"] = 1.05 #FOV in radians
transforms["w"] = 1280 # image width
transforms["h"] = 720 # image height


transforms["pixel_width"] = 2*math.tan(transforms["fov_x"]) * transforms["focal_length"] / transforms["w"] #Sensor width in mm
transforms["pixel_height"] = transforms["pixel_width"] #Sensor height in mm
transforms["fl_x"] = transforms["focal_length"]/transforms["pixel_width"] # focal length/ pixel_width
transforms["fl_y"] = transforms["focal_length"]/transforms["pixel_height"]# focal length/ pixel_height
transforms["cx"] = transforms["w"]//2 # optical center in x
transforms["cy"] = transforms["h"]//2 # optical center in y
transforms["camera_model"] = "OPENCV" # camera model

In [4]:
frames = []

# Radius of hemisphere
r = 1

cnt = 0

for i in range(0, 90, 15):
    for j in range(-180, 180, 15):
        transform_matrix = np.eye(4)

        # Translation
        transform_matrix[:3, 3] = [0, 0, r]

        # Rotation
        rot = Rotation.from_euler('xyz', [i, 0, j], degrees=True)
        rot_matrix = np.eye(4)
        rot_matrix[:3, :3] = rot.as_matrix()
        transform_matrix = np.matmul(rot_matrix, transform_matrix)

        frame = {}
        file_path = "images/{}.png".format(cnt)

        frame["file_path"] = file_path
        frame["transform_matrix"] = transform_matrix.tolist()

        frames.append(frame)
        cnt+=1

transforms["frames"] = frames

In [5]:
#Save transforms
with open(os.path.join(SAVE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)